In [8]:
%pwd



'/home/user/Desktop/medical_bot/Eye_Disease_Chatbot/research'

For geting back to root directory to load pdfs


In [9]:
import os
os.chdir("../")

%pwd

'/home/user/Desktop/medical_bot/Eye_Disease_Chatbot'

Load Pdfs

In [10]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
#extracting text from pdf files
def load_pdfs(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [12]:
extract_pdfs = load_pdfs("Data")

In [13]:
extract_pdfs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-12-10T12:55:31+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-12-10T12:55:31+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'Data/DME_KB_Document.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Diabetic Macular Edema (DME) — Comprehensive KB Entry\n1 — Short Definition\nDiabetic Macular Edema (DME) is a vision■threatening retinal condition caused by fluid accumulation\nwithin the macula due to breakdown of the blood–retinal barrier in diabetic retinopathy. It leads to\nintraretinal cysts, retinal thickening, and sometimes subretinal fluid, all visible on OCT.\n2 — Pathophysiology\n- Chronic hyperglycemia damages retinal capillary endothelial cells and pericytes.\n- Loss of pericytes → microaneurysms and capillary leakage.\n- Increased VEGF and inflammatory cytokines promote vasc

In [14]:
#filtering out empty documents
from typing import List
from langchain.schema import Document
def filter_minimal(docs: List[Document])-> List[Document]:
    """
    Given a list of Documents objects, retuern a new list of Document objects
    containing only {source} and {page_content} attributes.
    """
    filtered_docs : List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        filtered_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return filtered_docs

In [15]:
filtered_docs = filter_minimal(extract_pdfs)
filtered_docs

[Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='Diabetic Macular Edema (DME) — Comprehensive KB Entry\n1 — Short Definition\nDiabetic Macular Edema (DME) is a vision■threatening retinal condition caused by fluid accumulation\nwithin the macula due to breakdown of the blood–retinal barrier in diabetic retinopathy. It leads to\nintraretinal cysts, retinal thickening, and sometimes subretinal fluid, all visible on OCT.\n2 — Pathophysiology\n- Chronic hyperglycemia damages retinal capillary endothelial cells and pericytes.\n- Loss of pericytes → microaneurysms and capillary leakage.\n- Increased VEGF and inflammatory cytokines promote vascular permeability.\n- Breakdown of the inner blood–retinal barrier causes fluid entry into retinal layers.\n- Fluid accumulates as intraretinal cystoid spaces, spongiform thickening, and occasionally subretinal\nfluid.\n- Chronic DME results in photoreceptor layer disruption and poor visual prognosis.\n3 — Key OCT Imaging Features

In [16]:
import re

def clean_text(docs: List[Document]) -> List[Document]:
    """
    Clean page_content for all documents in the list.
    """
    cleaned_docs = []
    for doc in docs:
        # Remove non-ASCII characters and unwanted blocks
        cleaned = re.sub(r'[^\x00-\x7F]+', '', doc.page_content)  # Remove non-ASCII
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()  # Normalize whitespace
        if cleaned:  # Skip empty after cleaning
            cleaned_docs.append(Document(page_content=cleaned, metadata=doc.metadata))
    return cleaned_docs

# Apply to all filtered_docs
cleaned_docs = clean_text(filtered_docs)

In [17]:
cleaned_docs

[Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='Diabetic Macular Edema (DME) Comprehensive KB Entry 1 Short Definition Diabetic Macular Edema (DME) is a visionthreatening retinal condition caused by fluid accumulation within the macula due to breakdown of the bloodretinal barrier in diabetic retinopathy. It leads to intraretinal cysts, retinal thickening, and sometimes subretinal fluid, all visible on OCT. 2 Pathophysiology - Chronic hyperglycemia damages retinal capillary endothelial cells and pericytes. - Loss of pericytes microaneurysms and capillary leakage. - Increased VEGF and inflammatory cytokines promote vascular permeability. - Breakdown of the inner bloodretinal barrier causes fluid entry into retinal layers. - Fluid accumulates as intraretinal cystoid spaces, spongiform thickening, and occasionally subretinal fluid. - Chronic DME results in photoreceptor layer disruption and poor visual prognosis. 3 Key OCT Imaging Features (LayerLevel) - **Intraret

Chucking


In [18]:
#splitting documents into smaller chunks
def text_split(cleaned_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=45,
        chunk_overlap=12
    )
    text_chunks = text_splitter.split_documents(cleaned_docs)
    return text_chunks

In [19]:
text_chunks=text_split(cleaned_docs)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 501


In [20]:
text_chunks

[Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='Diabetic Macular Edema (DME) Comprehensive KB'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='KB Entry 1 Short Definition Diabetic Macular'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='Macular Edema (DME) is a visionthreatening'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='retinal condition caused by fluid'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='by fluid accumulation within the macula due'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='macula due to breakdown of the bloodretinal'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='barrier in diabetic retinopathy. It leads to'),
 Document(metadata={'source': 'Data/DME_KB_Document.pdf'}, page_content='It leads to intraretinal cysts, retinal'),
 Document(metadata={'source': 'Data/DME_KB_Document

Embeddings


In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
def text_embedding():
    """
    Download and Return HuggingFace Embeddings model.
    """
    Model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=Model_name

        )
    return embeddings

embeddings = text_embedding()

/tmp/ipykernel_13303/2696662282.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [52]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [53]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [24]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key) #for connecting to pinecone database, pc is picone connect

pc

In [25]:
pc.list_indexes()

[
    {
        "name": "eye-disease-chatbot",
        "metric": "cosine",
        "host": "eye-disease-chatbot-kuovyk1.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [26]:
# from pinecone import ServerlessSpec
# index_name = "eye-disease-chatbot"
# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=384, #Dimenstion of embedings , if you know Dimension you can directly pass it
#         metric="cosine", #for similarity search
#         spec=ServerlessSpec(
#             cloud = "aws",
#             region = "us-east-1"
#         )
#     )

# index = pc.Index(index_name)

In [27]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding=embeddings,
    index_name="eye-disease-chatbot"
)

In [28]:
restriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [30]:
restriver.invoke("what is Drusen?")

[Document(id='e6ac7d3c-8125-4d9f-b3cb-e437fa99d3f6', metadata={'source': 'Data/DRUSEN_KB_Document.pdf'}, page_content='articles and OCT-focused papers on drusen'),
 Document(id='8953fc52-e780-4112-8a38-b80bd50ab46b', metadata={'source': 'Data/DRUSEN_KB_Document.pdf'}, page_content='routine invasive treatment for drusen'),
 Document(id='fb2c9dcd-78fd-436f-8145-977821c9a29f', metadata={'source': 'Data/DRUSEN_KB_Document.pdf'}, page_content='Definition (clinician): "Drusen are sub-RPE')]

In [61]:
from langchain_google_genai import ChatGoogleGenerativeAI
ChatModel = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [62]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [72]:
system_prompt = (
    "You are a helpful medical assistant specializing in eye diseases. "
    "Answer questions based on the provided context from reliable medical sources. "
    "If the context doesn't fully cover the query, use your general knowledge to provide accurate, concise information. "
    "Be clear, concise, and empathetic. Always advise consulting a qualified eye care professional for diagnosis or treatment. if they ask for medical advaice. "
    "By the given context answer the questions carefully without menstioning that you are an AI language model. "
    "when they ask you about the any disease provide the all the information that you in the context and also include the general infromation form realbel medical sources. to extend the answer in clear way."
    "Do not provide medical advice or diagnoses. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])


In [73]:
question_answer_chain = create_stuff_documents_chain(
    llm=ChatModel,
    prompt=prompt 
)
rag_chain = create_retrieval_chain(restriver, question_answer_chain)

In [74]:
response = rag_chain.invoke({'input': "What is cnv?"})
print(response["answer"])

CNV stands for Choroidal Neovascularization.

Based on the provided context, CNV is described as a process in which abnormal blood vessels grow. These abnormal blood vessels originate from the choroid, which is the layer of blood vessels supplying blood to the retina, and extend into or under the retina.

These newly formed blood vessels are often fragile and can leak fluid or blood, which can significantly disrupt the normal function of the retina and lead to vision loss. CNV is a common and serious complication of various eye conditions, most notably age-related macular degeneration (AMD), but can also occur with other conditions like high myopia, inflammatory eye diseases, or trauma.

The context also mentions that CNV can be active or inactive. **Inactive CNV** refers to a state where there is an absence of fluid, though the presence of the abnormal vessels remains. Active CNV, conversely, would involve signs of leakage or bleeding.

If you have concerns about CNV or your eye healt

I don't have information on what DME stands for or what it is. Please consult a qualified eye care professional for diagnosis or treatment.
